지난번 퀴즈 리뷰 (예외처리!) 스파크 퀴즈 크롤링

In [1]:
from selenium import webdriver

# Step 1. 크롬 드라이버 위치 설정.
chr_driver_path = '/Applications/chromedriver'

# Step 2. 크롬 드라이버가 사용할 크롬 브라우저의 옵션을 설정.
options = webdriver.ChromeOptions()
# options.add_argument('window-size=5120x2880') -> 작동이 안 된다.

# Step 3. 위에 옵션을 활용하여 드라이버를 만든다.
driver = webdriver.Chrome(chr_driver_path, options=options)
# 혹시라도 창이 작아서 안 보이는 일이 없도록 최대화로 열기. -> 이걸 아래에 예외처리로 할거다.
# driver = webdriver.Chrome(chr_driver_path)
# driver.maximize_window()

# Step 4. 스파크 홈페이지로 이동하기.
spark_url = 'https://sparkkorea.com/'
driver.get(spark_url)    # URL을 가지고 이동하는 방법. driver.get()
# driver.current_url # 디버깅용.

In [2]:
# Step 5. 스파크 홈페이지 내에서 퀴즈로 이동하기.
spark_main_xpath = '//*[@id="site-navigation"]/button'
spark_quiz_xpath = '//*[@id="menu-item-382"]/a'

try:
    #(모바일 접속시 퀴즈 버튼이 바로 안 보임)
    driver.find_element_by_xpath(spark_main_xpath).click()    # 메뉴를 먼저 찾고
    driver.find_element_by_xpath(spark_quiz_xpath).click()    # 퀴즈 버튼을 찾아라
    print("모바일 접근")
except Exception as e:
    driver.find_element_by_xpath(spark_quiz_xpath).click()    # 퀴즈 버튼을 찾아라
    print("PC 접근")

driver.current_url # 디버깅용.

PC 접근


'https://sparkkorea.com/%ed%80%b4%ec%a6%88/'

# 공공 데이터 수집
### 1. 파일 데이터 수집

공공데이터포털에서 '주택금융관련 지수' - '지역별 PIR 및 LIR'을 불러와보자. '다운로드'를 누르면 csv파일이 다운로드가 되는데, 다운로드 누른 다음 그 다운로드 경로를 복사해 활용한다.

In [21]:
url = 'https://www.data.go.kr/cmm/cmm/fileDownload.do?atchFileId=FILE_000000001581679&fileDetailSn=1'

import pandas as pd
koreaPirLir = pd.read_csv(url, encoding='cp949')
koreaPirLir

# 어떤 지역 데이터가 있는지 컬럼 헤더를 조회해보자.
koreaPirLir.columns

# 서울 지역의 PIR 평균을 구해보자.
import numpy as np
np.average(koreaPirLir['서울PIR'])

## 2. OpenAPI JSON 데이터 수집

국가통계포털(KOSIS)에서 '행정구역(시군구)별, 성별 인구수'를 불러와보자. 활용 신청하고 URL 생성까지 해야한다.

### 방법 1. pd.json_normalize()

In [6]:
jsonUrl = 'http://kosis.kr/openapi/statisticsData.do?method=getList&apiKey=OGM2N2RlYTNmMjQ2MGY3NWEyYzg5MGU2NmJjYTI4Zjg=&format=json&jsonVD=Y&userStatsId=ox8eb1f9/101/DT_1B040A3/2/1/20200505230018&prdSe=M&newEstPrdCnt=1'

import pandas as pd
import requests

# 정상 여부 확인
response = requests.get(jsonUrl)
response

# JSON 데이터 획득
json = response.json()
# json

# 데이터프레임으로 저장
population = pd.json_normalize(json)
population.head()

,TBL_NM,PRD_DE,TBL_ID,ITM_NM,ITM_NM_ENG,ITM_ID,UNIT_NM,ORG_ID,UNIT_NM_ENG,C1_OBJ_NM,C1_OBJ_NM_ENG,DT,PRD_SE,C1,C1_NM,C1_NM_ENG
0,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,51842524,M,00,전국,Whole country
1,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,9726787,M,11,서울특별시,Seoul
2,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,3408347,M,26,부산광역시,Busan
3,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,2431523,M,27,대구광역시,Daegu
4,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,2950972,M,28,인천광역시,Incheon


### 방법 2. pd.read_json()

In [4]:
jsonUrl = 'http://kosis.kr/openapi/statisticsData.do?method=getList&apiKey=OGM2N2RlYTNmMjQ2MGY3NWEyYzg5MGU2NmJjYTI4Zjg=&format=json&jsonVD=Y&userStatsId=ox8eb1f9/101/DT_1B040A3/2/1/20200505230018&prdSe=M&newEstPrdCnt=1'

import pandas as pd
import requests

try:
    response = requests.get(jsonUrl)
    if response.status_code == 200:
        population = pd.read_json(jsonUrl)
    else:
        response.close()
except Exception as e:
    print(e)

population.head()

,TBL_NM,PRD_DE,TBL_ID,ITM_NM,ITM_NM_ENG,ITM_ID,UNIT_NM,ORG_ID,UNIT_NM_ENG,C1_OBJ_NM,C1_OBJ_NM_ENG,DT,PRD_SE,C1,C1_NM,C1_NM_ENG
0,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,51842524,M,0,전국,Whole country
1,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,9726787,M,11,서울특별시,Seoul
2,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,3408347,M,26,부산광역시,Busan
3,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,2431523,M,27,대구광역시,Daegu
4,행정구역(시군구)별 성별 인구수,202004,DT_1B040A3,총인구수,Koreans (Total),T20,명,101,Person,행정구역(시군구)별,By Administrative District,2950972,M,28,인천광역시,Incheon


## 3. OpenAPI XML 데이터 수집

공공데이터포털에서 '아파트매매 실거래 상세 자료'를 불러와보자.

In [7]:
# EndPoint
xmlUrl = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?_wadl&type=xml'

In [1]:
# Service URL
xmlUrl = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'

### 일반 방법

In [2]:
import requests, bs4
import pandas as pd
from lxml import html

In [3]:
# xml 데이터 불러오기.
response = requests.get(xmlUrl).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')    # 이번에는 'html.parser'가 아니라 'lxml-xml'이다.
# xmlobj # 디버깅용.

### 파라미터 분리 방법

In [4]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

In [5]:
# 파라미터 분리하기
My_API_Key = unquote('Agq7hySmyMi1FFU9kYibP+EnxYepQ/B6Dn+w9lsYKVSCDjTwIdvpjmuhJrtyQrhipg3F3a4jbSq/LxHi/dUIoQ==')    # 사용자 인증키

queryParams = '?' + urlencode(
    {
        quote_plus('ServiceKey') : My_API_Key,
        quote_plus('LAWD_CD'): '11110',
        quote_plus('DEAL_YMD'): '201512'
     }
)

In [6]:
xmlobj

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>99</resultCode><resultMsg>INVALID REQUEST PARAMETER ERROR.</resultMsg></header></response>

In [ ]:
http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev
    ?LAWD_CD=11110&DEAL_YMD=201512&serviceKey=AT33s775KYpJOkUBJu0dxkJuUeIfDIOJRzAH084EQS3JN%2BzFjErLHuk%2FGZa9L4gBTSGCzeA69tI9PwLp7B37IQ%3D%3D

In [ ]:
API_Key = unquote('sjOpJdbWHR3lhq4f8w5Dg6lc6IDZPdXeue%2BzGqY47h4Jp4sKy8SE5wKtpoHT7yfpI1NMFLyMMYHSiY4MNP6dhQ%3D%3D')
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'

In [ ]:
한국환경공단_미세먼지 경보 발령 현황
http://openapi.airkorea.or.kr/openapi/services/rest/UlfptcaAlarmInqireSvc

# 4.27 수업 분

In [11]:
import pandas as pd

In [12]:
population_url = 'https://www.data.go.kr/cmm/cmm/fileDownload.do?atchFileId=FILE_000000001581679&fileDetailSn=1'

In [13]:
pd.read_csv(population_url, encoding='cp949')

,연도,월,전국PIR,서울PIR,부산PIR,대구PIR,인천PIR,광주PIR,대전PIR,울산PIR,...,세종LIR,경기LIR,강원LIR,충북LIR,충남LIR,전북LIR,전남LIR,경북LIR,경남LIR,제주LIR
0,2004,3,4.21,4.89,3.95,3.73,4.65,2.81,4.68,2.66,...,-,2.42,1.44,1.53,1.21,1.90,1.42,1.31,2.06,1.28
1,2004,4,4.39,5.59,3.91,3.88,4.59,2.92,3.83,2.74,...,-,2.79,1.18,1.71,1.97,1.57,1.33,1.26,1.92,2.46
2,2004,5,4.19,5.14,4.90,3.83,4.78,3.41,4.19,2.93,...,-,2.22,1.44,1.72,1.85,1.75,1.31,1.50,1.85,2.14
3,2004,6,4.09,4.38,4.20,3.77,4.30,2.83,4.19,2.81,...,-,2.62,1.53,1.57,1.79,1.64,1.19,1.56,1.89,1.81
4,2004,7,4.29,5.88,4.43,4.28,4.81,3.32,4.35,2.54,...,-,2.41,1.68,1.91,2.15,1.64,1.36,1.50,1.93,2.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,2017,4,5.58,7.11,5.81,5.66,5.65,5.05,5.23,5.59,...,3.17,2.96,2.12,2.42,2.39,2.07,2.15,2.44,2.66,3.55
158,2017,5,5.69,7.13,5.73,5.93,5.58,5.18,5.45,5.24,...,3.17,2.99,2.22,2.25,2.39,2.31,2.08,2.18,2.80,3.23
159,2017,6,5.75,7.49,6.17,6.04,5.83,5.13,5.05,5.61,...,2.86,2.99,2.17,2.19,2.28,2.35,2.16,2.32,2.70,3.07
160,2017,7,5.79,7.87,6.13,5.91,5.83,4.94,5.10,5.24,...,3.25,3.09,2.49,2.14,2.21,2.33,1.90,2.60,2.54,2.86


공공데이터 포털 API 데이터 불러오기 (4-2 외부데이터 P.48)

In [14]:
rowList = []
columnList = []

columnList.append("82,500")
columnList.append("2008")
columnList

['82,500', '2008']

In [15]:
rowList.append(columnList)
rowList

[['82,500', '2008']]

In [16]:
columnList = [] # 다음 rows에 넣기 위해 비워준다.

In [17]:
columnList.append("60,000")
columnList.append("1982")
rowList.append(columnList)
columnList = []
rowList

[['82,500', '2008'], ['60,000', '1982']]

In [18]:
pd.DataFrame(rowList)

,0,1
0,"82,500",2008
1,"60,000",1982


P.49

In [ ]:
# 수업 예제 URL
http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc\
/getRTMSDataSvcAptTradeDev?LAWD_CD=11110&DEAL_YMD=201512&serviceKey=AT33s775KYpJOkUBJu0dxkJuUeIfDIOJRzAH084EQS3JN%2BzFjErLHuk%2FGZa9L4gBTSGCzeA69tI9PwLp7B37IQ%3D%3D

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import html

In [21]:
exampleUrl = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc\
/getRTMSDataSvcAptTradeDev?LAWD_CD=11110&DEAL_YMD=201512&serviceKey=AT33s775KYpJOkUBJu0dxkJuUeIfDIOJRzAH084EQS3JN%2BzFjErLHuk%2FGZa9L4gBTSGCzeA69tI9PwLp7B37IQ%3D%3D'

In [37]:
response = requests.get(exampleUrl)
response.encoding = 'utf-8'
xmlobj = BeautifulSoup(response.text,"lxml-xml")
# xmlobj # 디버깅용.

In [38]:
rowList = []
columnList = []
rows = xmlobj.findAll('item')
# rows # 디버깅용.

In [39]:
rowsLen = len(rows)
rowsLen

10

In [40]:
columns = rows[0].findAll()
columns

[<거래금액>    82,500</거래금액>,
 <건축년도>2008</건축년도>,
 <년>2015</년>,
 <도로명>사직로8길</도로명>,
 <도로명건물본번호코드>00004</도로명건물본번호코드>,
 <도로명건물부번호코드>00000</도로명건물부번호코드>,
 <도로명시군구코드>11110</도로명시군구코드>,
 <도로명일련번호코드>03</도로명일련번호코드>,
 <도로명지상지하코드>0</도로명지상지하코드>,
 <도로명코드>4100135</도로명코드>,
 <법정동> 사직동</법정동>,
 <법정동본번코드>0009</법정동본번코드>,
 <법정동부번코드>0000</법정동부번코드>,
 <법정동시군구코드>11110</법정동시군구코드>,
 <법정동읍면동코드>11500</법정동읍면동코드>,
 <법정동지번코드>1</법정동지번코드>,
 <아파트>광화문풍림스페이스본(101동~105동)</아파트>,
 <월>12</월>,
 <일>10</일>,
 <일련번호>11110-2203</일련번호>,
 <전용면적>94.51</전용면적>,
 <지번>9</지번>,
 <지역코드>11110</지역코드>,
 <층>11</층>]

In [41]:
columns[0].name

'거래금액'

In [42]:
columns[0].text

'    82,500'

In [44]:
columns[1].name

'건축년도'

In [45]:
columns[1].text

'2008'

In [46]:
rowList = []
columnList = []
nameList = []

In [47]:
for i in range(0, len(columns),):
    # 1. 컬럼값 저장
    columnList.append(columns[i].text)
rowList.append(columnList)    

In [32]:
pd.DataFrame(rowList)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,"82,500",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,11500,1,광화문풍림스페이스본(101동~105동),12,10,11110-2203,94.51,9,11110,11


In [55]:
for j in range(0, rowsLen):
    columns = rows[j].findAll()
    
    for i in range(0, len(columns),):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다.
        if j == 0:
            nameList.append(columns[i].name)
        # 1. 컬럼값 저장
        columnList.append(columns[i].text)
    rowList.append(columnList)
    columnList = []

In [60]:
pd.DataFrame(rowList).head()

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,"82,500",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,11500,1,광화문풍림스페이스본(101동~105동),12,10,11110-2203,94.51,9,11110,11
1,"82,500",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,11500,1,광화문풍림스페이스본(101동~105동),12,10,11110-2203,94.51,9,11110,11
2,"60,000",1981,2015,세종대로23길,00047,00000,11110,02,0,4100190,...,None,None,None,None,None,None,None,None,None,None
3,"130,000",2004,2015,경희궁2길,00005,00005,11110,01,0,4100005,...,None,None,None,None,None,None,None,None,None,None
4,"105,000",2004,2015,사직로8길,00024,00000,11110,05,0,4100135,...,None,None,None,None,None,None,None,None,None,None


In [57]:
for eachRow in rows:
    columns = eachRow.findAll()
    
    for eachColumns in columns:
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다.
        if j == 0:
            nameList.append(eachColumns.name)
        # 1. 컬럼값 저장
        columnList.append(eachColumns.text)
    rowList.append(columnList)
    columnList = []

In [59]:
pd.DataFrame(rowList).head()

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,"82,500",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,11500,1,광화문풍림스페이스본(101동~105동),12,10,11110-2203,94.51,9,11110,11
1,"82,500",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,11500,1,광화문풍림스페이스본(101동~105동),12,10,11110-2203,94.51,9,11110,11
2,"60,000",1981,2015,세종대로23길,00047,00000,11110,02,0,4100190,...,None,None,None,None,None,None,None,None,None,None
3,"130,000",2004,2015,경희궁2길,00005,00005,11110,01,0,4100005,...,None,None,None,None,None,None,None,None,None,None
4,"105,000",2004,2015,사직로8길,00024,00000,11110,05,0,4100135,...,None,None,None,None,None,None,None,None,None,None


P.50

In [1]:
# import requests, bs4
import requests
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

In [2]:
API_Key = unquote('key')
exampleUrl = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc\
/getRTMSDataSvcAptTradeDev?LAWD_CD=11110&DEAL_YMD=201512&serviceKey=AT33s775KYpJOkUBJu0dxkJuUeIfDIOJRzAH084EQS3JN%2BzFjErLHuk%2FGZa9L4gBTSGCzeA69tI9PwLp7B37IQ%3D%3D'

queryParams = '?' + urlencode(    # ? 뒤에 urlencode를 딕셔너리로 붙여놓은거다.
    {
        quote_plus('ServiceKye'):API_Key,
        quote_plus('LAWD_CD'):'11110',
        quote_plus('DEAL_YMD'):'201512'    
    }
)

In [3]:
response = requests.get(exampleUrl + queryParams)
response.encoding = 'UTF-8'
xmlobj = BeautifulSoup(response.text,'lxml-xml')
# xmlobj = bs4.BeautifulSoup(response.text,'lxml-xml')
xmlobj

<?xml version="1.0" encoding="utf-8"?>
<response><header><resultCode>99</resultCode><resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg></header></response>

In [66]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from bs4 import BeautifulSoup

API_Key = unquote('sjOpJdbWHR3lhq4f8w5Dg6lc6IDZPdXeue%2BzGqY47h4Jp4sKy8SE5wKtpoHT7yfpI1NMFLyMMYHSiY4MNP6dhQ%3D%3D')
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
queryParams = '?' + urlencode(
    {
        quote_plus('ServiceKey') : API_Key,
        quote_plus('LAWD_CD'): '11110',
        quote_plus('DEAL_YMD'): '201512'
     }
)

response = requests.get(url+queryParams)
response.encoding='utf-8'
xmlobj = BeautifulSoup(resp.text,"lxml-xml")

NameError: name 'resp' is not defined

In [4]:
rowList = []
columnList = []
rows = xmlobj.findAll('item')
# rows # 디버깅용.

In [5]:
columns = rows[3].findAll()
columns

IndexError: list index out of range

In [9]:
for i in range(0, len (columns)):
    rowList.append(columns[i].name)
    columnList.append(columns[i].text)
    rowList = rowList.append(columnList)

NameError: name 'columns' is not defined